In [1]:
import numpy as np
from sklearn import datasets
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [2]:
# Carga de datos.
datos = datasets.load_iris()
# print(datos)
print(np.shape(datos.data))

(150, 4)


In [3]:
# Partición EXTERNA. Test: hold-out split 80-20%.
X_train, X_test, y_train, y_test = train_test_split(datos.data, datos.target, test_size=0.2, random_state=42)
print('Train dimensions: ', np.shape(X_train))
print('Test dimensions:  ', np.shape(X_test))

Train dimensions:  (120, 4)
Test dimensions:   (30, 4)


In [4]:
# Algoritmo de aprendizaje.
clf = DummyClassifier(strategy='stratified', random_state=42)

In [5]:
# Validación y obtención de las predicciones del modelo.
# en "cv=KFold(n_splits=5)" se hace el cross-validation INTERNO!!
# el comando "cross_val_predict" nos permite extraer directamente las predicciones de las 5 bolsas de validación
y_pred = cross_val_predict(clf, X_train, y_train, cv = KFold(n_splits=5, shuffle=True, random_state=42))
print(y_pred)

[1 1 0 2 0 2 2 0 1 0 2 2 1 1 2 1 0 2 0 2 2 0 0 1 2 2 1 0 0 1 1 1 0 1 1 1 0
 0 1 2 2 1 1 1 2 2 1 1 0 1 2 2 2 2 2 2 1 0 1 1 2 1 2 2 0 2 0 1 1 1 2 1 1 2
 2 1 0 1 1 0 1 1 2 2 0 1 0 1 1 0 1 0 2 1 2 0 2 0 0 2 1 0 1 0 2 0 0 0 2 2 1
 1 0 1 0 0 1 1 1 2]


In [6]:
# Presentación de los resultados de la evaluación.
print("Exactitud: %.3f" % (metrics.accuracy_score(y_train, y_pred))) # accuracy
print("Precisión: %.3f" % (metrics.precision_score(y_train, y_pred, average="macro"))) # precision
print("Sensibilidad: %.3f" % (metrics.recall_score(y_train, y_pred, average="macro"))) # sensibilidad
print("F1-score: %.3f" % (metrics.f1_score(y_train, y_pred, average="macro"))) # F-score


Exactitud: 0.333
Precisión: 0.339
Sensibilidad: 0.333
F1-score: 0.335


In [7]:
# Extraemos la matriz de confusión
print("Matriz de confusión:\n", metrics.confusion_matrix(y_train, y_pred))

Matriz de confusión:
 [[13 19  8]
 [10 14 17]
 [11 15 13]]


In [8]:
print("Tabla de métricas:\n", metrics.classification_report(y_train, y_pred))

Tabla de métricas:
               precision    recall  f1-score   support

           0       0.38      0.33      0.35        40
           1       0.29      0.34      0.31        41
           2       0.34      0.33      0.34        39

    accuracy                           0.33       120
   macro avg       0.34      0.33      0.33       120
weighted avg       0.34      0.33      0.33       120



In [9]:
# Una vez analizado lo que ocurre en el conjunto de datos de validación, utilizamos todos los datos de "train" y "val"
# para construir un modelo definitivo
clf = clf.fit(X_train, y_train) # Entrenamos el modelo

# Y posteriormente evaluar el resultado del modelo entrenado con el conjunto de test
test_results = clf.score(X_test, y_test) # Evaluamos el test
print('Exactitud en test: ', np.round(test_results*100,4), '%')

Exactitud en test:  36.6667 %


In [10]:
# También podemos obtener las predicciones del test para extraer el resto de métricas de interés sobre el test
y_pred_test = clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred_test))
print(metrics.confusion_matrix(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.56      0.50      0.53        10
           1       0.31      0.44      0.36         9
           2       0.25      0.18      0.21        11

    accuracy                           0.37        30
   macro avg       0.37      0.38      0.37        30
weighted avg       0.37      0.37      0.36        30

[[5 3 2]
 [1 4 4]
 [3 6 2]]


In [11]:
# Veamos qué ocurre con un algoritmo más complejo que "dummyclassifier"

# Hacemos el cross-validation INTERNO para seleccionar los mejores hiperparámetros
from sklearn.svm import SVC
svc = SVC(C=0.5)
y_pred = cross_val_predict(svc, X_train, y_train, cv = KFold(n_splits=5, shuffle=True, random_state=42))

print(metrics.classification_report(y_train, y_pred))
print(metrics.confusion_matrix(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.86      0.93      0.89        41
           2       0.92      0.85      0.88        39

    accuracy                           0.93       120
   macro avg       0.93      0.92      0.92       120
weighted avg       0.93      0.93      0.92       120

[[40  0  0]
 [ 0 38  3]
 [ 0  6 33]]


In [12]:
# Una vez hemos optimizado los hiperparámetros con el conjunto de validación, utilizamos todos los datos de 
# "train" y "validation" para entrenar un modelo definitivo con esos hiperparámetros ajustados
svc = SVC(C=0.5) # Definimos el modelo con los hiperparámetros óptimos
svc = svc.fit(X_train, y_train) # Entrenamos el modelo con todos los datos de "train" + "val"

# y finalmente evaluamos el test
test_results = svc.score(X_test, y_test) # Evaluamos la exactitud del test
print('Exactitud en test: ', np.round(test_results*100,4), '%')

Exactitud en test:  100.0 %


In [13]:
# obtenemos las predicciones del test para extraer el resto de métricas de interés
y_pred_test = svc.predict(X_test)
print(metrics.classification_report(y_test, y_pred_test))
print(metrics.confusion_matrix(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

[[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
